https://universaldependencies.org/

In [ ]:
!pip install DAWG stanza multiprocessing

In [ ]:
!pip install stanza

# Разбор синтаксиса

In [ ]:
import glob
import os
import stanza
from ipywidgets import IntProgress
from IPython.display import display, clear_output
import multiprocessing
import time
import torch

def get_filenames():
    return glob.iglob('/mnt/notebooks-a/home/kukuruku/data/proza_ru/**/*.txt', recursive=True)


FILES_COUNT = 10000000


def process_text(p):
#     progress = IntProgress(min=0, max=FILES_COUNT) # instantiate the bar
#     display(progress)
    
    skip_num = p * 30000
    retries = 0
    
    time.sleep(p*30)
    print(f"{p} starts")
    
    while True:
        try:
            stanza.download('ru', processors="tokenize,pos,lemma,depparse")
            nlp = stanza.Pipeline('ru', processors="tokenize,pos,lemma,depparse")
                
            break
        except Exception:
            retries += 1
            time.sleep(p*2 + 10)
            
            if retries > 10:
                print(f"{p} cannot load nlp")
                return
            
            print('Retry load nlp')


    i = 0
    print(f"{p} work")

    with torch.no_grad():
        for file_name in get_filenames():
            i += 1
            if i < skip_num: continue

            stanza_file_name = f"{file_name}.stanza"

            if os.path.isfile(stanza_file_name):
                continue

            if i % 100 == 0:
                print(p, i)

    #         progress.value = i
    #         progress.description = f"p {p}: {i} of {FILES_COUNT}"
            try:
               with open(file_name, 'r') as file:
                    with open(stanza_file_name, 'wb') as stanza_file:
                        text = file.read()
                        doc = nlp(text)
                        stanza_file.write(doc.to_serialized())
            except Exception as e:
                print(e)
                torch.cuda.empty_cache()
                continue

 


# if __name__ == '__main__':
#     print('Main tread: start workers')
#     jobs = []
#     for i in range(4):
#         p = multiprocessing.Process(target=process_text, args=(i,))
#         jobs.append(p)
#         p.start()

process_text(0)


0 starts


2022-03-03 07:55:46 INFO: Downloading these customized packages for language: ru (Russian)...
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| pretrain  | syntagrus |

2022-03-03 07:55:46 INFO: File exists: /home/kukuruku/stanza_resources/ru/tokenize/syntagrus.pt.
2022-03-03 07:55:46 INFO: File exists: /home/kukuruku/stanza_resources/ru/pos/syntagrus.pt.
2022-03-03 07:55:46 INFO: File exists: /home/kukuruku/stanza_resources/ru/lemma/syntagrus.pt.
2022-03-03 07:55:48 INFO: File exists: /home/kukuruku/stanza_resources/ru/depparse/syntagrus.pt.
2022-03-03 07:55:49 INFO: File exists: /home/kukuruku/stanza_resources/ru/pretrain/syntagrus.pt.
2022-03-03 07:55:49 INFO: Finished downloading models and saved to /home/kukuruku/stanza_resources.
2022-03-03 07:55:49 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | synta

0 work
0 81300


# Чтерие готовых данных

In [ ]:
import glob
import os
import stanza
from ipywidgets import IntProgress
from IPython.display import display, clear_output

if not 'nlp' in locals():
    stanza.download('ru')
    nlp = stanza.Pipeline('ru')
    clear_output(wait=True)
    
    
file_names = glob.iglob('/home/kukuruku/data/proza_ru/**/*.txt', recursive=True)
    
    
for file_name in file_names:
    stanza_file_name = f"{file_name}.stanza"
    
    try:   
        with open(stanza_file_name, 'rb') as stanza_file:
            doc = stanza.Document.from_serialized(stanza_file.read())
            
            print(doc)
    except Exception:
        break
    

# Распознание сущностей

In [ ]:
import stanza
from IPython.display import clear_output


if not 'nlp' in locals():
    stanza.download('ru')
    nlp = stanza.Pipeline('ru')
    clear_output(wait=True)

    

def find_root_entity(words):
    root = None
    amods = []
    
    for w in words:
        if w.upos == "NOUN" and w.deprel == "root":
            root = w
            break
            
    if not root:
        return None
    
    for w in words:
        if w.deprel == "amod" and w.head == root.id:
            amods.append(w)

    return ' '.join([*[a.text for a in amods], root.text])


assert find_root_entity(nlp('белая курица').sentences[0].words) == 'белая курица'
assert find_root_entity(nlp('рябина красная').sentences[0].words) == 'красная рябина'

nlp('застежка липучка')
# assert find_root_entity(nlp('застежка липучка').sentences[0].words) == 'застежка липучка'


import dawg

words = ['hello', 'world', 'hello world', 'hello dawg']

base_dawg = dawg.DAWG(words)
completion_dawg = dawg.CompletionDAWG(words)